In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Classification with Generative Models on Vertex AI

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

Generative models like PaLM 2 are powerful language models used for various natural language processing (NLP) tasks. One of those is text classification, which involves assigning one or more categories to a given piece of text. Although text classification can be done using traditional NLP techniques, LLMs can perform classification by providing prompts (as opposed to domain-specific labeled data), which can accelerate the time it takes to build a text classification solution. Classification models based on LLMs can be further tuned with many examples via custom model training, but that is beyond the scope of this notebook.

In this notebook, you will explore how to do text classification using prompts with the PaLM API. Learn more about classification prompts in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/classification-prompts).

### Objective

By the end of the notebook, you should be able to  use a large language model to perform various classification tasks, including:

* Zero-shot prompting text classification
* Few-shot prompting text classification
* Common tasks:
    * Sentiment analysis
    * Topic classification
    * Spam detection
    * Intent recognition
    * Language identification
    * Toxicity detection
    * Emotion detection

## Getting Started

### Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top. 

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

**Colab only:** Uncomment the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.  

In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import pandas as pd
from vertexai.preview.language_models import TextGenerationModel

### Import models

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

### Create the translation wrapper function

In [ ]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

## Text Classification

In the section below, you will explore zero-shot prompting, few-shot prompting, and some common types of text classification tasks.

### Zero-shot prompting

Zero-shot prompting is where you do not provide examples with labels, and rely on the LLM to make the classification on its own.

In [ ]:
prompt = traduza("""
Classifique a texto abaixo:\n
texto: "Hoje eu vi um animal peludo no parque - Ele tinha um rabo logo e grandes olhos."
label: cachorros, gatos
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "pt")
)

### Few-shot prompting

With few-shot prompting, you provide examples to the PaLM model and expect it to predict classes based on the provided examples.

In [ ]:
prompt = traduza("""
Qual é o assunto de uma determinada manchete de notícias? \n
- negócios \n
- entretenimento \n
- saúde \n
- esportes \n
- tecnologia \n\n

Texto: revisão prática do especialista do Pixel 7 Pro. \n
A resposta é: tecnologia \n

Texto: Parar de fumar? \n
A resposta é: saúde \n

Texto: Passarinhos ou bichos-papões? As 5 principais dicas para bater abaixo do par \n
A resposta é: esportes \n

Texto: Alívio do aumento do salário mínimo local parece mais remoto \n
A resposta é: negócios \n

Texto: Você não vai adivinhar quem acabou de chegar a Bari, na Itália, para a estreia do filme. \n
A resposta é:
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "pt")
)

### Other classification examples

Explore some more common text classification prompts below, which are all based on zero-shot prompts. You can also turn some of these into few-shot prompts by providing your own custom examples of text and the associated output classes.

#### Topic classification

In [ ]:
prompt = traduza("""
Classifique um trecho de texto em um dos vários tópicos predefinidos, como esportes, política ou entretenimento. \n
texto: O presidente Biden visitará a Índia no mês de março para discutir algumas oportunidades. \n
classe:
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "pt")
)

####  Spam detection

In [ ]:
prompt = traduza("""
Dado um e-mail, classifique-o como spam ou não spam. \n
e-mail: oi usuário, \n
       você foi selecionado como vencedor da loteria e pode ganhar até 1 milhão de dólares. \n
       gentilmente compartilhe seus dados bancários e podemos prosseguir a partir daí. \n\n

       de, \n
       Departamento oficial de loteria dos EUA
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "pt")
)

#### Intent recognition

In [ ]:
prompt = traduza("""
Dada a entrada de um usuário, classifique sua intenção, como "encontrar informações", "fazer uma reserva" ou "fazer um pedido". \n
entrada do usuário: Olá, você pode agendar uma mesa para dois no Restaurante Google para 1º de maio?
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "pt")
)

#### Language identification

In [ ]:
prompt = traduza("""
Dado um pedaço de texto, classifique o idioma em que está escrito. \n
texto: Selam nasıl gidiyor?
linguagem:
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "pt")
)

#### Toxicity detection

In [ ]:
prompt = traduza("""
Dado um trecho de texto, classifique-o como tóxico ou não tóxico. \n
texto: eu amo dias ensolarados
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text, "pt")
)

#### Emotion detection

In [ ]:
prompt = traduza("""
Dado um trecho de texto, classifique a emoção que ele transmite, como 'felicidade' ou 'raiva'. \n
texto: Estou muito feliz com as notícias de ontem
""", "en")

print(
    traduza(generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.5).text, "pt")
)

### Evaluation

You can evaluate the outputs of the text classification task if the ground truth classes are available. To showcase how this works, start by creating a simple dataframe with product reviews and the ground truth sentiment.

In [ ]:
review_data = {
    "review": [
        "adoro este produto. Tem tudo o que procuro!",
        "tudo o que posso dizer é que você ficará feliz depois de comprar este produto",
        "é muito caro e não vale o preço",
        "Ele é ok. Não é bom nem muito ruim.",
    ],
    "sentiment_groundtruth": ["positivo", "positivo", "negativo", "neutro"],
}

review_data_df = pd.DataFrame(review_data)
review_data_df

Now that you have the data with reviews and sentiments as ground truth labels, you can call the text generation model to each review row using the `apply` function. Each row will use the prompt in the `review` column to predict the sentiment using the PaLM API, and store the results in `sentiment_prediction` column.  

In [ ]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" and "negative". \n\n
                review: {traduza(row, "en")} \n
                sentiment:
              """
    response = generation_model.predict(prompt=prompt).text
    return traduza(response, "pt")


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

In the end, you can call the `classification_report` function from sklearn to measure the accuracy and other classification metrics by passing ground truth sentiments `sentiment_groundtruth` and predicted sentiment `sentiment_prediction`:

In [ ]:
from sklearn.metrics import classification_report

print(
    classification_report(
        review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
    )
)